# NOTEBOOK FOURRE-TOUT pour scrips PlaMADE

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import os, re
import pandas as pd
import geopandas as gp
import numpy as np
import zipfile
import Connexion_Transfert as ct
import Outils as O
from geoalchemy2 import Geometry,WKTElement
from shapely.geometry import Point, LineString
from shapely.geometry.point import Point
from shapely.geometry.multipoint import MultiPoint

c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\geopandas\_compat.py:84: UserWarning: The Shapely GEOS version (3.4.3-CAPI-1.8.3 r4285) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


In [2]:
from shapely import speedups
speedups.disable()

In [ ]:
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees'
#dézipper tous les dossier de Gérard
for root,dirs, files in os.walk(dossierSrc) : 
    for f in files : 
        if f.endswith('.zip') : 
            cheminFichier=os.path.join(root, f)
            print(cheminFichier[:-4])
            try:
                with zipfile.ZipFile(cheminFichier) as z:
                    z.extractall(cheminFichier[:-4])
                    print(f"fichier extrait {cheminFichier[:-4]}")
            except:
                print(f"pb extraction sur dossier {cheminFichier[:-4]}")

## 1. creer une bdd des fichiers geostandardises

In [ ]:
#pour traiter tous les fichiers d'un dossier général.suppose que la structure des tables a déjà été crée (par exemple avec l'import des des fichiers pouis truncate)
dossierSrc=r'D:\Boulot\PlaMADE\Ile-de-france\75-Paris\75'
coupleFichierTable=(('N_ROUTIER_ALLURE','allure_national'),('N_ROUTIER_REVETEMENT','rvt_national'),('N_ROUTIER_ROUTE','route_national'),
                    ('N_ROUTIER_TRAFIC','trafic_national'),('N_ROUTIER_VITESSE','vts_national'))
#dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne-Rhone-Alpes\Donnees_geostandardisees\Route01_v2_dec2020'
listEreur=[]
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    for root,dirs, files in os.walk(dossierSrc) : 
        for f in files :
            if f.endswith(('.shp', '.dbf')) : 
                print(os.path.join(root,f))
                if f.endswith('.shp') and 'N_ROUTIER_TRONCON' in f :
                    try :
                        ct.ogr2ogr_shp2pg(c.connstringOgr,os.path.join(root,f),
                                              schema='geostandardise_src', table='troncon_national',
                                              SRID=None,geotype='MULTILINESTRINGZ', dims=3, creationMode='-append -update',encodageClient='UTF-8', version_simple=True)
                    except Exception as e: 
                        listEreur.append({'fichier': f, erreur : e})
                else : 
                    for fich,t in coupleFichierTable :
                        try : 
                            if fich in f.upper() and f.endswith('.dbf') : 
                                ct.ogr2ogr_shp2pg(c.connstringOgr,os.path.join(root,f),
                                                  schema='geostandardise_src', table=t, SRID=None,geotype=None, dims=None, creationMode='-append -update',encodageClient='UTF-8', requeteSql='', version_simple=True)
                            elif fich in f.upper() and f.endswith('.csv') :
                                df = pd.read_csv(os.path.join(root,f), 
                                                 keep_default_na=False)
                                df.columns=[c.lower() for c in df.columns]
                                df.drop(colonnesEnTrop,axis=1).to_sql(t,c.sqlAlchemyConn,'geostandardise_src', if_exists='append', index=False )
                        except Exception as e: 
                            listEreur.append({'fichier': f, 'erreur' : e})

In [ ]:
#si les donnees trafic, vts, allure , rvt sont en csv, le plus simple c'est pandas : 
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne'
coupleFichierTable=(('N_ROUTIER_ALLURE','allure_national'),('N_ROUTIER_REVETEMENT','rvt_national'),('N_ROUTIER_ROUTE','route_national'),
                    ('N_ROUTIER_TRAFIC','trafic_national'),('N_ROUTIER_VITESSE','vts_national'))
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    for root,dirs, files in os.walk(dossierSrc) : 
            for f in files :
                if f.endswith('.csv') :
                    for fich,t in coupleFichierTable :
                        if fich in f.upper() : 
                            try : 
                                print(os.path.join(root,f))
                                df = pd.read_csv(os.path.join(root,f), 
                                                keep_default_na=False)
                                df.columns=[c.lower() for c in df.columns]
                                dfref=pd.read_sql(f'select * from geostandardise_src.{t} limit 1',c.sqlAlchemyConn)
                                colonnesEnTrop=[c.lower() for c in df.columns if c not in dfref.columns]
                                df.drop(colonnesEnTrop,axis=1).to_sql(t,c.sqlAlchemyConn,'geostandardise_src', if_exists='append', index=False )
                            except Exception as e :
                                print(f'Erreur sur : {os.path.join(root,f)} : {e} ')

In [ ]:
#pour test sur un dept
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    ct.ogr2ogr_shp2pg(c.connstringOgr,r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne\35\N_ROUTIER_TRONCON_L_035.shp',
                                              schema='geostandardise_src', table='troncon_national',
                                              SRID='2154',geotype='MULTILINESTRING', dims=3, creationMode='-append -update',encodageClient='UTF-8')

In [ ]:
#sur un fichier
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    ct.ogr2ogr_shp2pg(c.connstringOgr,r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne_Rhone_Alpes\Donnees_geostandardisees\Route01_v2_dec2020\N_ROUTIER_ALLURE_001.dbf',
                                                  schema='geostandardise_src', table='allure_national', SRID=None,geotype=None, dims=None, creationMode='-append -update',encodageClient='UTF-8', requeteSql='', version_simple=True)


## 2. Générer un fichier unique trafic RD
> L'idée est de se baser sur le fichier de résumé des fichiers gestionnaires et de leurs attributs pour produire un fichier concaténé, avec des attributs uniques relatifs au :
- tmja
- pcpl
- nom de la voie
- annee du trafic
- fichier source (sur Box internet)
- nom de la source

> ### 2.1 Principe mise en forme des fichiers 
>> Pour faire ça on va aller chercher le [fichier de synthses des données RD] (https://cerema.app.box.com/folder/132749694470/Synthese_type_fichier.ods).
dans l'onglet trafic, on va itérer sur chaque ligne (donc récupération du tuple de valeur) : 
>> On limite l'analyse aux données présentant un colonne valide='oui' & type='sig' & type_geom='ligne'  
1. on récupere la valeur de la colonne "nom_fichier_trafic" :
    1. si l'extenstion est présente, on va lire le fichier (attention, via box Drive, donc paramètre de raw string en entrée)
    1. l'attribut relatif au tmja est toujours présent, celui au pc_pl parfois, comme pour les routes ou l'année, mais comme la fonction rename s'en fout, on fait le rename des 4 colonnes
    1. si des formules sont à appliquer, on les applique (liées au pcpl, et nom de route notamment
    1. gérer le format des dates pour l'année de mesure
    1. ajouter les attributs sur le fichiers sources, le nom de la source, le type de source
    1. enregistrer le fichier mise en forme dans le dossier qui va bien (paramètre entrée)

In [270]:
# ouvrir le fichiers de syntheses des données
dossierDonneesRd=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD'
fichierSynthese=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD\Synthese_type_fichier.ods'
dfFichierSynthese=pd.read_excel(fichierSynthese, sheet_name='Complet',engine='odf', nrows=35, dtype={'dept':str})

In [ ]:
#creer le generateurde parcours des valeusr en tuple
dfNettoyeesSansReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='ligne')
                                 & (dfFichierSynthese['regles_gestion']=='non')].copy()
dfNettoyeesAvecReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='ligne')
                                 & (dfFichierSynthese['regles_gestion']!='non')].copy()
dfNettoyees=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='ligne')].copy()

In [8]:
#fonction d'ajout des sources de données
def ajouterSourceDonnees(tupleNomAttr, dfFichierRenomme, fihcierSource) : 
    """
    ajouter les nomsource, fichier source et type de source aux donnees mise en formes
    in : 
        tupleNomAttr : tuple des noms d'attributs recherche dans le fchier source
        dfFichierRenomme : gdf : contient la trsucture attributaire prevue, apres calcul
        fihcierSource : String : nom du fichier source
    """
    dfFichierRenomme['nomsource']='CD_'+tupleNomAttr.dept
    dfFichierRenomme['fichie_src']=fihcierSource

In [7]:
#fonction d'ouverture et reprojection en 2154
def ouvrirReprojeter(cheminFichier, tupleNomAttr):
    """
    tupleNomAttr : tuple des noms d'attributs recherche dans le fchier source
    """
    dfFichierSource=gp.read_file(cheminFichier, crs=tupleNomAttr.projection)
    dfFichierSource=dfFichierSource.to_crs('epsg:2154')
    return dfFichierSource
    

In [293]:
#fonction de calcul regroupant les traitements propres a chaque dept ayant des regles de gestion

def calculSpecifique(dfFichierSource, t, listAttrNonNull, typeAtr='trafic') : 
    """
    calculer les valeur de tmja, pcpl, nom route et année dans les cas particuklier ou des regles de gestion sont necessaire
    in : 
        dfFichierSource : gdf : gdf issue des fichiers osurces gestionnaiere
        t : tuple des noms d'attributs recherche dans le fchier source
        listAttrNonNull : list des attributs present dans le fichier (ayant une valuer non nulle)
        typeAtr : str : 'trafic' ou 'vts' ou'rvt' defaukt 'trafic' : traduit le type de thématique du fichier ouvert
    out : 
        dfFichierRenomme : gdf : contient la trsucture attributaire prevue, apres calcul
    """
    dicoRvt={'ECF':['ECF','Réparations localisées','Pavés','Bicouche','GLG bicouche','PAVES','Enrobé Coulé à Froid','ENROBE COULE A FROID','ECF cl. A Bicouche 0/10','Pavés','ENROBES COULES A FROID',
                   'ECF cl. A Bicouche 0/6,3'],
             'BBSG': ['Béton bitumineux','BBSG','Inversé','BBSG (10% recycle)','BBSG (10%)','BBSG (20%)','BBSG (30%)','BBSG (BT)','Colbifibre','COLBIFIBRE','RCS','REPRO','RSC',
                     'Enrobés','AUTRE','EB10','Béton Bitumineux Autre','Béton Bitumineux Froid','EB10 Tiède','Revêtement Haute Adhérence','Non connu','Non Revêtu',
                     'PATA','CALIBRAGE','ENROBE','REPROFILAGE','?','-','BB','BBF','BBS','BBSG','EME','GB','TN18','X','BB','BBF', 'ENROBE SEMI-GRENU', 'ENROBES'],
             'ES' : ['ESU','Produits Spéciaux','Monocouche DG','Monocouche MG','ESGL','ESGLg','ESGLG','ESLG','ESLG 6/10','ESLGg','ESLGl','ESLGLg','Enduit','Enduit Superficiel',
                    'ENDUIT SUPERFICIEL','ESEPAIS','AC','ES','ESBIC','ESGLG','ESMDG','ESMONO', 'ESU', 'ENDUITS SUPERFICIELS', 'REPROFILAGE'],
             'BBME' : ['BBME','BBME (10% recycle)', 'ENROBE MINCE A MODULE ELEVE',],
             'BBM' : ['BBS', 'BBM','BBMA','BBMA phonique','BBMA PHONIQUE', 'BBMPHO','ENROBE MINCE','BBMa', 'BBMa '],
             'BBTM' : ['BBTM','BBHM','Béton Bitumineux Très Mince','BB Très Mince Acoustique', 'BBTMPHO','ENROBE TRES MINCE'],
            'BBUM': ['BBUM','ULTRA MINCE','ENROBE ULTRA MINCE'],
            'BBDr':['BBDR',],
            'BC':['BC','BBC']}
    
    def defRvt(rvt):
        if rvt :
            try : 
                rvtOk=[k for k, v in dicoRvt.items() if rvt in v][0]
            except IndexError : 
                print(f'ATTENTION {rvt}')
                return None
        else :
            return None
        return rvtOk
 
    if t.dept=='012' : 
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : x['TMJATV2015'] if not pd.isnull(x['TMJATV2015']) else x['TMJA_TV'], axis=1)
        dfFichierSource['annee']=dfFichierSource.apply(lambda x : re.match('2[0-9]{3]',x['COMMENTAIR']) if not pd.isnull(x['COMMENTAIR']) else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='002': 
        dfFichierSource['rvtType']=dfFichierSource.apply(lambda x : defRvt(x['NatureCouc']) if not pd.isnull(x['NatureCouc']) else None, axis=1)
        dfFichierSource['anneePose']=dfFichierSource.apply(lambda x : str(x.AnneeCouch) if not x.AnneeCouch[0]=='<' else str(x.AnneeCouch)[1:], axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute',t.nom_attr_annee:'annee',t.nom_attr_trafic:'trafic',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_granulo:'granulo'})
    elif t.dept=='025' :
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : x['TMJA'] if x['%_PL']!='D' else None, axis=1)
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : str(x['%_PL']) if x['%_PL']!='D' else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_route:'nomRoute', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='027' :
        dfFichierSource['pcpl']=dfFichierSource.F_PL.apply(lambda x : x.replace('%','').replace(',','.') if not pd.isnull(x) else None)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic', t.nom_attr_annee:'annee',t.nom_attr_route:'nomRoute',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='031' :
        dfFichierSource['nomRoute']=dfFichierSource.route.apply(lambda x : O.epurationNomRoute(x))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',t.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='037' :
        if typeAtr=='vts' :
            dfFichierSource['nomRoute']=dfFichierSource.axe.apply(lambda x : x.split('_')[1])
        elif  typeAtr=='rvt' :
            dfFichierSource['nomRoute']=dfFichierSource.IDROUTE.apply(lambda x : x.split('_')[1])
            dfFichierSource['rvtType']=dfFichierSource.CG_TYPECOU.apply(lambda x : defRvt(x))
        else :
            dfFichierSource['nomRoute']=dfFichierSource.idroute.apply(lambda x : x.split('_')[1])
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',t.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='046': 
        dfFichierSource['trafic']=dfFichierSource['MJA TV (an'].apply(lambda x : x.replace(' ','').split('(')[0] if not pd.isnull(x) else None)
        dfFichierSource['annee']=dfFichierSource['MJA TV (an'].apply(lambda x : x.replace(' ','').split('(')[1][:-1] if not pd.isnull(x) and '(' in x else None)
        dfFichierSource['pcpl']=dfFichierSource['%PL'].apply(lambda x : float(x)*100 if not pd.isnull(x) else None)
        dfFichierSource['nomRoute']=dfFichierSource['RD'].apply(lambda x : x.replace(' ','').replace('RD','D'))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='049' :
        dfFichierSource['nomRoute']=dfFichierSource.Route.apply(lambda x : O.epurationNomRoute(x[3:]))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',t.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='052' :
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : str(x['MJA_PL']/x['MJA_TV']*100) if not pd.isnull(x['MJA_TV']) else None, axis=1)
        dfFichierSource['nomRoute']=dfFichierSource.Route.apply(lambda x : O.epurationNomRoute(x))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='058' :
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : x['TRAFFIC_VL']+x['TRAFFIC_PL'], axis=1)
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : str(x['TRAFFIC_PL']/(x['TRAFFIC_VL']+x['TRAFFIC_PL'])*100) if x['TRAFFIC_VL']+x['TRAFFIC_PL']>0 else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_route:'nomRoute', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='059' :
        dfFichierSource['nomRoute']=dfFichierSource.VOIE.apply(lambda x : O.epurationNomRoute(x[1:]))
        dfFichierSource['rvtType']=dfFichierSource.apply(lambda x : defRvt(x['TYP_DE_REV']) if not pd.isnull(x['TYP_DE_REV']) else defRvt(x['C_D_C_DE_S']), axis=1)
        dfFichierSource['granulo']=dfFichierSource.apply(lambda x : re.match(r'([0-9]{1,2}/[0-9]{1,2})',x.GRANULOMET)[0] if not pd.isnull(x.GRANULOMET) and re.match(r'([0-9]/[0-9])',x.GRANULOMET) else None, axis=1)
        dfFichierSource['anneePose']=dfFichierSource.apply(lambda x : str(x.D_D_C_DE_S)[:4] if not pd.isnull(x.D_D_C_DE_S) else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',t.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl'})
    elif t.dept=='060' :
        if  typeAtr=='trafic' :
            dfFichierSource['pcpl']=dfFichierSource.PCT_PL_SEM.apply(lambda x : x.replace('%PL','') if not x=='%PL' else None)
            dfFichierSource['annee']=dfFichierSource.DATE_COMPT.str[-4:]
        elif typeAtr=='rvt': 
            dfFichierSource['rvtType']=dfFichierSource.NATURE_C_R.apply(lambda x : defRvt(x.split('_')[0] if not pd.isnull(x) else None))
            dfFichierSource['granulo']=dfFichierSource.apply(lambda x : re.match(r'([0-9]{1,2}/[0-9]{1,2})',x.NATURE_C_R)[0] if not pd.isnull(x.NATURE_C_R) and re.match(r'([0-9]/[0-9])',x.NATURE_C_R) else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',t.nom_attr_route:'nomRoute',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_annee_pose : 'anneePose'})
    if t.dept=='067' :
        if typeAtr=='rvt': 
            dfFichierSource['rvtType']=dfFichierSource.REVET.apply(lambda x : defRvt(x))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',t.nom_attr_route:'nomRoute',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_annee_pose : 'anneePose',t.nom_attr_revetement:'granulo'}) 
    if t.dept=='068' : 
        if typeAtr=='rvt': 
            dfFichierSource['rvtType']=dfFichierSource.ROULEMENT_.apply(lambda x : defRvt(x))
            dfFichierSource['granulo']=dfFichierSource.apply(lambda x : re.match(r'([0-9]{1,2}/[0-9]{1,2})',x.ROULEMEN_1)[0] if not pd.isnull(x.ROULEMEN_1) and re.match(r'([0-9]/[0-9])',x.ROULEMEN_1) else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',t.nom_attr_route:'nomRoute',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_annee_pose : 'anneePose'})             
    elif t.dept=='070':
        for c in [f'TA_TV{i}' for i in range(10,19)]: 
            dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c])>0 if not pd.isnull(x[c]) else False, axis=1),'trafic']= dfFichierSource[c]
            dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c.replace('TV','PL')])>0 if not pd.isnull(x[c.replace('TV','PL')]) else False, axis=1),'pcpl']=(dfFichierSource.loc[dfFichierSource.apply(
                lambda x : float(x[c.replace('TV','PL')])>0 if not pd.isnull(x[c.replace('TV','PL')]) else False, axis=1)].apply(lambda x : float(x[c.replace('TV','PL')]), axis=1)/
              dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c.replace('TV','PL')])>0 if not pd.isnull(x[c.replace('TV','PL')]) else False, axis=1)].apply(lambda x : float(x[c]), axis=1)*100)                
            dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c])>0 if not pd.isnull(x[c]) else False , axis=1),'annee']= f'20{c[-2:]}'
            dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_route:'nomRoute',
                                                            t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='075' :
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : (x['MT']*12)+(x['ME']*4)+(x['MN']*8) if not (pd.isnull(x['MT']) and pd.isnull(x['ME']) and pd.isnull(x['MN'])) else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept =='080' :
        dfFichierSource['nomRoute']=dfFichierSource[t.nom_attr_route].str[3:]
        dfFichierSource['rvtType']=dfFichierSource.apply(lambda x : defRvt(x['NATURE']) if not pd.isnull(x['NATURE']) else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',t.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='082' :
        dfFichierSource['nomRoute']=dfFichierSource[t.nom_attr_route].str[3:]
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',t.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept =='076' : 
        dfFichierSource['nomRoute']=dfFichierSource[t.nom_attr_route].apply(lambda x : O.epurationNomRoute(x[3:]) if x[:2]!='VC' else x )
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',t.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='077' :
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : str(x['PLTMJA']/x['TVTMJA']*100) if not pd.isnull(x['TVTMJA']) else None, axis=1)
        dfFichierSource['nomRoute']=dfFichierSource.ROUTE.apply(lambda x : O.epurationNomRoute(x[2:]))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic', t.nom_attr_annee:'annee',
                                                        t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    elif t.dept=='083' :
        dfFichierSource['nomRoute']=dfFichierSource.ROUTE.apply(lambda x : O.epurationNomRoute(x[3:]))
        if typeAtr=='vts' :
            dfFichierSource['vtsvl']=dfFichierSource.PARAMETRES.apply(lambda x : float(x[10:12]) if not pd.isnull(x) else None)
        elif typeAtr=='trafic' :        
            dfFichierSource['trafic']=dfFichierSource.apply(lambda x : int(x['MJA_2018']) if int(x['MJA_2018'])>0 else int(x['MJA_2017']), axis=1)
            dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : x['PL_2018'] if int(x['MJA_2018'])>0 else str(int(x['PL_2017'])/10), axis=1)
            dfFichierSource['annee']=dfFichierSource.apply(lambda x : '2018' if int(x['MJA_2018'])>0 else '2017', axis=1)       
        else : 
            dfFichierSource['rvtType']=dfFichierSource.apply(lambda x : defRvt(x.MATERIAU) if not pd.isnull(x.MATERIAU) else defRvt(x.REVETEMENT), axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'})
    return dfFichierRenomme[listAttrNonNull]

In [ ]:
#Concat des donnees 
dicoGdf={}
for t in dfNettoyees.itertuples() : 
    #ouvrir le fichier
    print(t.dept)
    for root, dirs, files in os.walk(os.path.join(dossierDonneesRd, t.dept)) : 
        for f in files : 
            """if t.dept  not in  ('027') : 
                continue"""
            if f.lower() == t.nom_fichier_trafic.lower() : 
                #ouvrir et modifier le crs si besoin
                dfFichierSource=ouvrirReprojeter(os.path.join(root, f),t)
                dicoNomAttr={t.nom_attr_trafic:'trafic', t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute',t.nom_attr_annee:'annee'}
                listAttrNonNull=[v for k,v in dicoNomAttr.items() if not pd.isnull(k)]+['geometry']
                #print(listAttrNonNull)
                #a reprendre apres pour le cas des departements particuliers non traites
                if t.dept in ('012','027', '037', '049', '052', '058', '059', '060', '075', '077','082', '083', '076','080'):
                    dfFichierRenomme=calculSpecifique(dfFichierSource, t, listAttrNonNull)
                else :
                    dfFichierRenomme=dfFichierSource.rename(columns=dicoNomAttr)[listAttrNonNull]
                ajouterSourceDonnees(t, dfFichierRenomme, f)
    dicoGdf[t.dept]=dfFichierRenomme
dfConcat=pd.concat(dicoGdf.values())

In [418]:
# traitemnets avant export : 
#tagguer les geoms nulles 
dfConcat.loc[dfConcat.geometry.apply(lambda x : not pd.isnull(x)), 'geom_nulle']=False
dfConcat.loc[dfConcat.geometry.apply(lambda x : pd.isnull(x)), 'geom_nulle']=True

#corriger des valeurs louches sur les annees et passer en format date AAAA pour le reste
dfConcat.annee=dfConcat.annee.apply(lambda x : str(x) if not pd.isnull(x) else None)
dfConcat.annee=dfConcat.annee.str[:4]
dfConcat.loc[dfConcat.annee.apply(lambda x : float(x)<1950 if not (pd.isnull(x)) else True), 'annee']=None

#mise en forme valeurs TMJA et pcpl : 
dfConcat.trafic=dfConcat.trafic.apply(lambda x : round(float(x),0) if not pd.isnull(x) else np.nan)
dfConcat.pcpl=dfConcat.pcpl.apply(lambda x : round(float(str(x).replace(',','.')),2) if not pd.isnull(x) else np.nan)

In [419]:
#export
dfConcat.to_file(r'D:\Boulot\AffairesEnCours\plamade\RDs\concat\RD_concatenation_trafic_lineaire.shp')

> ### 2.2 Principe mise en forme des fichiers ponctuels
>> on va garder la structure précédente, mais il va aussi falloir ajouter les 2 jeu de données en tableur

In [405]:
dfNettoyeesSansReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='point')
                                 & (dfFichierSynthese['regles_gestion']=='non')].copy()
dfNettoyeesAvecReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='point')
                                 & (dfFichierSynthese['regles_gestion']!='non')].copy()
dfNettoyees=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='point')].copy()

In [ ]:
dicoGdf={}
for t in dfNettoyees.itertuples() : 
    #ouvrir le fichier
    print(t.dept)
    for root, dirs, files in os.walk(os.path.join(dossierDonneesRd, t.dept)) : 
        for f in files : 
            """if t.dept  not in  ('046') : 
                continue"""
            if f.lower() == t.nom_fichier_trafic.lower() : 
                #print(f)
                #ouvrir et modifier le crs si besoin
                dfFichierSource=ouvrirReprojeter(os.path.join(root, f),t)
                dicoNomAttr={t.nom_attr_trafic:'trafic', t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute',t.nom_attr_annee:'annee'}
                listAttrNonNull=[v for k,v in dicoNomAttr.items() if not pd.isnull(k)]+['geometry']
                #a reprendre apres pour le cas des departements particuliers non traites
                if t.dept in ('025','031', '070', '046'):
                    dfFichierRenomme=calculSpecifique(dfFichierSource, t, listAttrNonNull)
                else :
                    dfFichierRenomme=dfFichierSource.rename(columns=dicoNomAttr)[listAttrNonNull]
                ajouterSourceDonnees(t, dfFichierRenomme, f)
    dicoGdf[t.dept]=dfFichierRenomme
dfConcatFixe=pd.concat(dicoGdf.values())

In [408]:
# traitemnets avant export : 
#tagguer les geoms nulles 
dfConcat=dfConcatFixe.copy()

dfConcat.loc[dfConcat.geometry.apply(lambda x : not pd.isnull(x)), 'geom_nulle']=False
dfConcat.loc[dfConcat.geometry.apply(lambda x : pd.isnull(x)), 'geom_nulle']=True

#corriger des valeurs louches sur les annees et passer en format date AAAA pour le reste
dfConcat.annee=dfConcat.annee.apply(lambda x : str(x) if not pd.isnull(x) else None)
dfConcat.annee=dfConcat.annee.str[:4]
dfConcat.loc[dfConcat.annee.apply(lambda x : float(x)<1950 if not (pd.isnull(x)) else True), 'annee']=None
dfConcat.loc[dfConcat.nomsource=='CD_008', 'annee']='2018'

#mise en forme valeurs TMJA et pcpl : 
dfConcat.trafic=dfConcat.trafic.apply(lambda x : round(float(x),0) if not pd.isnull(x) else np.nan)
dfConcat.pcpl=dfConcat.pcpl.apply(lambda x : round(float(str(x).replace(',','.')),2) if not pd.isnull(x) else np.nan)

#toutes les geoms en multi
dfConcat["geometry"] = [MultiPoint([feature]) if isinstance(feature, Point) else feature for feature in dfConcat["geometry"]]

In [409]:
#export
dfConcat.to_file(r'D:\Boulot\AffairesEnCours\plamade\RDs\concat\RD_concatenation_trafic_ponctuel.shp')

## 3. Générer un fichier unique RD  
sur la base de ce qui a été fait au dessus, on ajouter le revetement et la vitesse quand ils sont disponible au sein du même fichier source, pour les lignes puis les points.  
ensuite il faut voir pour les fihiers multiples, on va plutot essayer de regrouper l'info au sein d'un seul fichier prétraité qui regroupera toute les infos

> ### 3.1 fichier lignes avec un seul fichier source et tous les attributs

In [273]:
dossierDonneesRd=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD'
fichierSynthese=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD\Synthese_type_fichier.ods'
dfFichierSynthese=pd.read_excel(fichierSynthese, sheet_name='Complet',engine='odf', nrows=35, dtype={'dept':str})

In [274]:
dfNettoyeesSansReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide_trafic']=='oui') & (dfFichierSynthese['type_trafic']=='sig') & (dfFichierSynthese['type_geom_trafic']=='ligne')
                                 & (dfFichierSynthese['regles_gestion_trafic']=='non') & (dfFichierSynthese['fichier_unique']=='oui')].copy()
dfNettoyeesAvecReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide_trafic']=='oui') & (dfFichierSynthese['type_trafic']=='sig') & (dfFichierSynthese['type_geom_trafic']=='ligne')
                                 & (dfFichierSynthese['regles_gestion_trafic']!='non') & (dfFichierSynthese['fichier_unique']=='oui')].copy()
dfNettoyees=dfFichierSynthese.loc[(dfFichierSynthese['valide_trafic']=='oui') & (dfFichierSynthese['type_trafic']=='sig') & (dfFichierSynthese['type_geom_trafic']=='ligne')
                                 & (dfFichierSynthese['fichier_unique']=='oui')].copy()

In [275]:
dicoGdf={}
for t in dfNettoyees.itertuples() : 
    #ouvrir le fichier
    print(t.dept)
    for root, dirs, files in os.walk(os.path.join(dossierDonneesRd, t.dept)) : 
        if t.dept  in  ('037','067', '068','083') : 
                continue
        for f in files : 
            if f.lower() == t.nom_fichier_trafic.lower() : 
                #print(f)
                #ouvrir et modifier le crs si besoin
                dfFichierSource=ouvrirReprojeter(os.path.join(root, f),t)
                dicoNomAttr={t.nom_attr_trafic:'trafic', t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute',t.nom_attr_annee:'annee',
                             t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'}
                listAttrNonNull=[v for k,v in dicoNomAttr.items() if not pd.isnull(k)]+['geometry']
                #a reprendre apres pour le cas des departements particuliers non traites
                if t.dept in ('012','027', '037', '049', '052', '058', '059', '060', '075', '077','082', '083', '076','080'):
                    dfFichierRenomme=calculSpecifique(dfFichierSource, t, listAttrNonNull)
                else :
                    dfFichierRenomme=dfFichierSource.rename(columns=dicoNomAttr)[listAttrNonNull]
                ajouterSourceDonnees(t, dfFichierRenomme, f)
    dicoGdf[t.dept]=dfFichierRenomme
dfConcatFixe=pd.concat(dicoGdf.values())

012
013
027
035
036
044
045
048
049
050
052
058
059
066
071
075
076
077
080
082


In [276]:
# traitemnets avant export : 
dfConcat=dfConcatFixe.copy()
#tagguer les geoms nulles 
dfConcat.loc[dfConcat.geometry.apply(lambda x : not pd.isnull(x)), 'geom_nulle']=False
dfConcat.loc[dfConcat.geometry.apply(lambda x : pd.isnull(x)), 'geom_nulle']=True

#corriger des valeurs louches sur les annees et passer en format date AAAA pour le reste
dfConcat.annee=dfConcat.annee.apply(lambda x : str(x) if not pd.isnull(x) else None)
dfConcat.annee=dfConcat.annee.str[:4]
dfConcat.loc[dfConcat.annee.apply(lambda x : float(x)<1950 if not (pd.isnull(x)) else True), 'annee']=None

#mise en forme valeurs TMJA et pcpl : 
dfConcat.trafic=dfConcat.trafic.apply(lambda x : round(float(x),0) if not pd.isnull(x) else np.nan)
dfConcat.pcpl=dfConcat.pcpl.apply(lambda x : round(float(str(x).replace(',','.')),2) if not pd.isnull(x) else np.nan)

#passage des NaN sur rvtType, granulo, anneePose en None
dfConcat.loc[pd.isnull(dfConcat.rvtType), 'rvtType']=None
dfConcat.loc[pd.isnull(dfConcat.granulo), 'granulo']=None
dfConcat.loc[pd.isnull(dfConcat.anneePose), 'anneePose']=None

#coller la granulo sur les codes Enumeres
def granuloStandard(granulo) : 
    dico_granulo={'0/4' : ['0/4',],
                    '0/6' : ['0/6',],
                    '0/8' : ['0/8',],
                    '0/10 type 1' : ['0/10','0/10 type 1'],
                    '0/10 type 2' : ['0/10 type 2',],
                    '0/14' : ['0/14',],
                    '4/6' : ['4/6',],
                    '6/8' : ['6/8','5/8'],
                    '6/10' : ['6/10'],
                    '10/14': ['10/14']}
    if granulo : 
        granuloOk=[k for k, v in dico_granulo.items() if granulo in v][0]
    else : 
        return None
    return granuloOk
dfConcat['granulo']=dfConcat.granulo.apply(lambda x : granuloStandard(x) )

In [277]:
#exemple de verifs
dfConcat.granulo.unique()
dfConcat.anneePose.unique()
#dfConcat.rvtType.unique()

array([None, '2017', '2014', '2009', '2004', '1990', '2011', '2012',
       '2016', '2015', '2007', '2005', '2010', '2018', '2000', '2008',
       '2002', '2006', '1997', '2003', '2019', '1984', '2001', '1998',
       '1995', '1999', '2013', '1994', '1991', '1985', '1996', '1950',
       '1978', '1987', '1993', '1960'], dtype=object)

In [278]:
#export
dfConcat.to_file(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\Bruit\PLAMADE\Donnees_GT_flash\donnees_produites\Concat_RDs\Concat_RDs_tousAttributs_1Fichier_lineaire.shp')

> ### 3.2 fichier point avec un seul fichier source et tous les attributs (il n' y a pas de fichiers point multiples pour plusieurs attributs)

In [279]:
dossierDonneesRd=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD'
fichierSynthese=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD\Synthese_type_fichier.ods'
dfFichierSynthese=pd.read_excel(fichierSynthese, sheet_name='Complet',engine='odf', nrows=35, dtype={'dept':str})

In [280]:
dfNettoyeesSansReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide_trafic']=='oui') & (dfFichierSynthese['type_trafic']=='sig') & (dfFichierSynthese['type_geom_trafic']=='point')
                                 & (dfFichierSynthese['regles_gestion_trafic']=='non')].copy()
dfNettoyeesAvecReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide_trafic']=='oui') & (dfFichierSynthese['type_trafic']=='sig') & (dfFichierSynthese['type_geom_trafic']=='point')
                                 & (dfFichierSynthese['regles_gestion_trafic']!='non')].copy()
dfNettoyees=dfFichierSynthese.loc[(dfFichierSynthese['valide_trafic']=='oui') & (dfFichierSynthese['type_trafic']=='sig') & (dfFichierSynthese['type_geom_trafic']=='point')].copy()

In [281]:
dicoGdf={}
for t in dfNettoyees.itertuples() : 
    #ouvrir le fichier
    print(t.dept)
    for root, dirs, files in os.walk(os.path.join(dossierDonneesRd, t.dept)) : 
        for f in files : 
            """if t.dept  not in  ('046') : 
                continue"""
            if f.lower() == t.nom_fichier_trafic.lower() : 
                #print(f)
                #ouvrir et modifier le crs si besoin
                dfFichierSource=ouvrirReprojeter(os.path.join(root, f),t)
                dicoNomAttr={t.nom_attr_trafic:'trafic', t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute',t.nom_attr_annee:'annee',
                            t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl', t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'}
                listAttrNonNull=[v for k,v in dicoNomAttr.items() if not pd.isnull(k)]+['geometry']
                #a reprendre apres pour le cas des departements particuliers non traites
                if t.dept in ('002','025','031', '070', '046'):
                    dfFichierRenomme=calculSpecifique(dfFichierSource, t, listAttrNonNull)
                else :
                    dfFichierRenomme=dfFichierSource.rename(columns=dicoNomAttr)[listAttrNonNull]
                ajouterSourceDonnees(t, dfFichierRenomme, f)
    dicoGdf[t.dept]=dfFichierRenomme
dfConcatFixe=pd.concat(dicoGdf.values())

002
008
025
031
039
046
070
089


In [282]:
# traitemnets avant export : 
dfConcat=dfConcatFixe.copy()
#tagguer les geoms nulles 
dfConcat.loc[dfConcat.geometry.apply(lambda x : not pd.isnull(x)), 'geom_nulle']=False
dfConcat.loc[dfConcat.geometry.apply(lambda x : pd.isnull(x)), 'geom_nulle']=True

#corriger des valeurs louches sur les annees et passer en format date AAAA pour le reste
dfConcat.annee=dfConcat.annee.apply(lambda x : str(x) if not pd.isnull(x) else None)
dfConcat.annee=dfConcat.annee.str[:4]
dfConcat.loc[dfConcat.annee.apply(lambda x : float(x)<1950 if not (pd.isnull(x)) else True), 'annee']=None

#mise en forme valeurs TMJA et pcpl : 
dfConcat.trafic=dfConcat.trafic.apply(lambda x : round(float(x),0) if not pd.isnull(x) else np.nan)
dfConcat.pcpl=dfConcat.pcpl.apply(lambda x : round(float(str(x).replace(',','.')),2) if not pd.isnull(x) else np.nan)

#passage des NaN sur rvtType, granulo, anneePose en None
dfConcat.loc[pd.isnull(dfConcat.rvtType), 'rvtType']=None
#dfConcat.loc[pd.isnull(dfConcat.granulo), 'granulo']=None
dfConcat.loc[pd.isnull(dfConcat.anneePose), 'anneePose']=None

#toutes les geoms en multi
dfConcat["geometry"] = [MultiPoint([feature]) if isinstance(feature, Point) else feature for feature in dfConcat["geometry"]]

#coller la granulo sur les codes Enumeres
def granuloStandard(granulo) : 
    dico_granulo={'0/4' : ['0/4',],
                    '0/6' : ['0/6',],
                    '0/8' : ['0/8',],
                    '0/10 type 1' : ['0/10','0/10 type 1'],
                    '0/10 type 2' : ['0/10 type 2',],
                    '0/14' : ['0/14',],
                    '4/6' : ['4/6',],
                    '6/8' : ['6/8','5/8'],
                    '6/10' : ['6/10'],
                    '10/14': ['10/14']}
    if granulo : 
        granuloOk=[k for k, v in dico_granulo.items() if granulo in v][0]
    else : 
        return None
    return granuloOk
#dfConcat['granulo']=dfConcat.granulo.apply(lambda x : granuloStandard(x) )

In [283]:
#exemple de verifs
#dfConcatFixe.granulo.unique()
#dfConcatFixe.anneePose.unique()
dfConcat.rvtType.unique()

array(['BBSG', 'ES', nan], dtype=object)

In [285]:
#export
dfConcat.to_file(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\Bruit\PLAMADE\Donnees_GT_flash\donnees_produites\Concat_RDs\Concat_RDs_tousAttributs_1Fichier_ponctuel.shp')

> ### 3.3 fichier lineaire avec un plusiuers fichier source ; attributs trafic

In [286]:
dossierDonneesRd=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD'
fichierSynthese=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD\Synthese_type_fichier.ods'
dfFichierSynthese=pd.read_excel(fichierSynthese, sheet_name='Complet',engine='odf', nrows=35, dtype={'dept':str})

In [287]:
dfNettoyeesSansReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide_trafic']=='oui') & (dfFichierSynthese['type_trafic']=='sig') & (dfFichierSynthese['type_geom_trafic']=='ligne')
                                 & (dfFichierSynthese['regles_gestion_trafic']=='non') & (dfFichierSynthese['fichier_unique']=='non')].copy()
dfNettoyeesAvecReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide_trafic']=='oui') & (dfFichierSynthese['type_trafic']=='sig') & (dfFichierSynthese['type_geom_trafic']=='ligne')
                                 & (dfFichierSynthese['regles_gestion_trafic']!='non') & (dfFichierSynthese['fichier_unique']=='non')].copy()
dfNettoyees=dfFichierSynthese.loc[(dfFichierSynthese['valide_trafic']=='oui') & (dfFichierSynthese['type_trafic']=='sig') & (dfFichierSynthese['type_geom_trafic']=='ligne') 
                                 & (dfFichierSynthese['fichier_unique']=='non')].copy()

In [288]:
dicoGdf={}
for t in dfNettoyees.itertuples() : 
    #ouvrir le fichier
    print(t.dept)
    for root, dirs, files in os.walk(os.path.join(dossierDonneesRd, t.dept)) : 
        for f in files : 
            if f.lower() == t.nom_fichier_trafic.lower() : 
                #print(f)
                #ouvrir et modifier le crs si besoin
                dfFichierSource=ouvrirReprojeter(os.path.join(root, f),t)
                dicoNomAttr={t.nom_attr_trafic:'trafic', t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute',t.nom_attr_annee:'annee'}
                listAttrNonNull=[v for k,v in dicoNomAttr.items() if not pd.isnull(k)]+['geometry']
                #a reprendre apres pour le cas des departements particuliers non traites
                if t.dept in ('012','027', '037', '049', '052', '058', '059', '060', '075', '077','082', '083', '076','080'):
                    dfFichierRenomme=calculSpecifique(dfFichierSource, t, listAttrNonNull)
                else :
                    dfFichierRenomme=dfFichierSource.rename(columns=dicoNomAttr)[listAttrNonNull]
                ajouterSourceDonnees(t, dfFichierRenomme, f)
    dicoGdf[t.dept]=dfFichierRenomme
dfConcatFixe=pd.concat(dicoGdf.values())

037
060
067
068
083


In [289]:
# traitemnets avant export : 
dfConcat=dfConcatFixe.copy()
#tagguer les geoms nulles 
dfConcat.loc[dfConcat.geometry.apply(lambda x : not pd.isnull(x)), 'geom_nulle']=False
dfConcat.loc[dfConcat.geometry.apply(lambda x : pd.isnull(x)), 'geom_nulle']=True

#corriger des valeurs louches sur les annees et passer en format date AAAA pour le reste
dfConcat.annee=dfConcat.annee.apply(lambda x : str(x) if not pd.isnull(x) else None)
dfConcat.annee=dfConcat.annee.str[:4]
dfConcat.loc[dfConcat.annee.apply(lambda x : float(x)<1950 if not (pd.isnull(x)) else True), 'annee']=None

#mise en forme valeurs TMJA et pcpl : 
dfConcat.trafic=dfConcat.trafic.apply(lambda x : round(float(x),0) if not pd.isnull(x) else np.nan)
dfConcat.pcpl=dfConcat.pcpl.apply(lambda x : round(float(str(x).replace(',','.')),2) if not pd.isnull(x) else np.nan)

In [290]:
#export
dfConcat.to_file(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\Bruit\PLAMADE\Donnees_GT_flash\donnees_produites\Concat_RDs\Concat_RDs_1Attributs_1Fichier_trafic_lineaire.shp')

> ### 3.3 fichier lineaire avec un plusiuers fichier source ; attributs vitesse

In [294]:
dicoGdf={}
for t in dfNettoyees.itertuples() : 
    #ouvrir le fichier
    print(t.dept)
    for root, dirs, files in os.walk(os.path.join(dossierDonneesRd, t.dept)) : 
        for f in files : 
            if isinstance(t.nom_fichier_vitesse, str) : 
                #print(f.lower(), t.nom_fichier_vitesse.lower())
                if f.lower() == t.nom_fichier_vitesse.lower() : 
                    #print(f)
                    #ouvrir et modifier le crs si besoin
                    dfFichierSource=ouvrirReprojeter(os.path.join(root, f),t)
                    dicoNomAttr={t.nom_attr_routevitesse:'nomRoute',t.nom_attr_vtsvl:'vtsvl', t.nom_attr_vtspl:'vtspl'}
                    listAttrNonNull=[v for k,v in dicoNomAttr.items() if not pd.isnull(k)]+['geometry']
                    #print(listAttrNonNull)
                    #a reprendre apres pour le cas des departements particuliers non traites
                    if t.dept in ('037','083'):
                        dfFichierRenomme=calculSpecifique(dfFichierSource, t, listAttrNonNull, ('vts'))
                    else :
                        dfFichierRenomme=dfFichierSource.rename(columns=dicoNomAttr)[listAttrNonNull]
                    ajouterSourceDonnees(t, dfFichierRenomme, f)
    dicoGdf[t.dept]=dfFichierRenomme
dfConcatFixe=pd.concat(dicoGdf.values())

037
060
067
068
083


In [295]:
# traitemnets avant export : 
dfConcat=dfConcatFixe.copy()
#ajout attribut vts pl pour homogénéïté
dfConcat['vtspl']=None

In [296]:
#exemple de vérif
dfConcat

,nomRoute,vtsvl,geometry,nomsource,fichie_src,vtspl
0,D1,70.0,"LINESTRING (542436.788 6704593.203, 542447.300...",CD_037,Lim_Vitesse_CD37.shp,None
1,D1,70.0,"LINESTRING (543306.486 6704744.750, 543312.500...",CD_037,Lim_Vitesse_CD37.shp,None
2,D1,50.0,"LINESTRING (551145.358 6707598.390, 551167.200...",CD_037,Lim_Vitesse_CD37.shp,None
3,D1,70.0,"LINESTRING (538365.946 6703752.032, 538418.700...",CD_037,Lim_Vitesse_CD37.shp,None
4,D1,70.0,"LINESTRING (538900.102 6703933.786, 538915.600...",CD_037,Lim_Vitesse_CD37.shp,None
...,...,...,...,...,...,...
357,EV8,50.0,"LINESTRING (932374.172 6282189.852, 932381.677...",CD_083,ARRETE_LIMITATION_VITESSE_ROUTE83_ORL.shp,None
358,EV8,30.0,"LINESTRING (932846.179 6282184.907, 932856.994...",CD_083,ARRETE_LIMITATION_VITESSE_ROUTE83_ORL.shp,None
359,EV8,30.0,"LINESTRING (938426.166 6282211.992, 938436.133...",CD_083,ARRETE_LIMITATION_VITESSE_ROUTE83_ORL.shp,None
360,EV8,30.0,"LINESTRING (940457.766 6280295.614, 940465.388...",CD_083,ARRETE_LIMITATION_VITESSE_ROUTE83_ORL.shp,None


In [297]:
#export
dfConcat.to_file(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\Bruit\PLAMADE\Donnees_GT_flash\donnees_produites\Concat_RDs\Concat_RDs_1Attributs_1Fichier_trafic_vitesse.shp')

> ### 3.3 fichier lineaire avec un plusiuers fichier source ; attributs revetement

In [298]:
dicoGdf={}
for t in dfNettoyees.itertuples() : 
    #ouvrir le fichier
    print(t.dept)
    for root, dirs, files in os.walk(os.path.join(dossierDonneesRd, t.dept)) : 
        for f in files : 
            if isinstance(t.nom_fichier_revetement, str) : 
                #print(f.lower(), t.nom_fichier_vitesse.lower())
                if f.lower() == t.nom_fichier_revetement.lower() : 
                    #print(f)
                    #ouvrir et modifier le crs si besoin
                    dfFichierSource=ouvrirReprojeter(os.path.join(root, f),t)
                    dicoNomAttr={t.nom_attr_route_rvt:'nomRoute',t.nom_attr_revetement:'rvtType',t.nom_attr_granulo:'granulo', t.nom_attr_annee_pose : 'anneePose'}
                    listAttrNonNull=[v for k,v in dicoNomAttr.items() if not pd.isnull(k)]+['geometry']
                    print(listAttrNonNull,dicoNomAttr )
                    #a reprendre apres pour le cas des departements particuliers non traites
                    if t.dept in ('037','060','067','068','083',):
                        dfFichierRenomme=calculSpecifique(dfFichierSource, t, listAttrNonNull, ('rvt'))
                    else :
                        dfFichierRenomme=dfFichierSource.rename(columns=dicoNomAttr)[listAttrNonNull]
                    ajouterSourceDonnees(t, dfFichierRenomme, f)
    dicoGdf[t.dept]=dfFichierRenomme
dfConcatFixe=pd.concat(dicoGdf.values())

037
['nomRoute', 'rvtType', 'granulo', 'anneePose', 'geometry'] {'IDROUTE': 'nomRoute', 'CG_TYPECOU': 'rvtType', 'CG_TYPEG_1': 'granulo', 'CGDT_MODIF': 'anneePose'}
060
['nomRoute', 'rvtType', 'geometry'] {'AXE': 'nomRoute', 'NATURE_C_R': 'rvtType', nan: 'anneePose'}
067
['nomRoute', 'rvtType', 'geometry'] {'AXE': 'nomRoute', 'REVET': 'rvtType', nan: 'anneePose'}
068
['nomRoute', 'rvtType', 'granulo', 'anneePose', 'geometry'] {'AXE': 'nomRoute', 'ROULEMENT_': 'rvtType', 'ROULEMEN_1': 'granulo', 'ANNEE': 'anneePose'}
083
['nomRoute', 'rvtType', 'anneePose', 'geometry'] {'ROUTE': 'nomRoute', 'REVETEMENT, MATERIAU': 'rvtType', nan: 'granulo', 'ANNEE': 'anneePose'}


In [299]:
# traitemnets avant export : 
dfConcat=dfConcatFixe.copy()

#corriger des valeurs louches sur les annees et passer en format date AAAA pour le reste
dfConcat.anneePose=dfConcat.anneePose.apply(lambda x : str(x) if not pd.isnull(x) else None)
dfConcat.anneePose=dfConcat.anneePose.str[:4]
dfConcat.loc[dfConcat.anneePose.apply(lambda x : float(x)<1950 if not (pd.isnull(x)) else True), 'annee']=None

#passage des NaN sur rvtType, granulo, anneePose en None
dfConcat.loc[pd.isnull(dfConcat.rvtType), 'rvtType']=None
dfConcat.loc[pd.isnull(dfConcat.granulo), 'granulo']=None
dfConcat.loc[pd.isnull(dfConcat.anneePose) | (dfConcat.anneePose=='0'), 'anneePose']=None

#coller la granulo sur les codes Enumeres
def granuloStandard(granulo) : 
    dico_granulo={'0/4' : ['0/4','2/4'],
                    '0/6' : ['0/6','0/5'],
                    '0/8' : ['0/8',],
                    '0/10 type 1' : ['0/10','0/10 type 1', '-', '10/1','/100', '0/25'],
                    '0/10 type 2' : ['0/10 type 2',],
                    '0/14' : ['0/14','0/20', '0/31'],
                    '4/6' : ['4/6', '6/4', '4/06'],
                    '6/8' : ['6/8','5/8'],
                    '6/10' : ['6/10','4/10'],
                    '10/14': ['10/14']}
    try :
        if granulo : 
            granuloOk=[k for k, v in dico_granulo.items() if granulo in v][0]
        else : 
            return None
    except IndexError : 
        print(granulo)
    return granuloOk
dfConcat['granulo']=dfConcat.granulo.apply(lambda x : granuloStandard(x) )

In [300]:
#exemple verifs
dfConcat.rvtType.unique()
dfConcat.granulo.unique()

array([None, '0/10 type 1', '4/6', '0/6', '6/10', '10/14', '0/4', '0/14',
       '0/8', '6/8'], dtype=object)

In [301]:
#export
dfConcat.to_file(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\Bruit\PLAMADE\Donnees_GT_flash\donnees_produites\Concat_RDs\Concat_RDs_1Attributs_1Fichier_Revetement_lineaire.shp')